# Chaigning Video Generations

This notebook will help you quickly start generating longer form videos using Amazon Nova Reel by stitching together multiple 6 second generations.

## Prerequisites

Be sure you've followed the instructions in the [00_initial_setup.ipynb](../00_initial_setup.ipynb) notebook to get things set up.


## Configure credentials and shared parameters

Run the cell below to set the default session configuration for all AWS SDK calls made by the other cells in this notebook. As written, the code will default to using the user credentials you have set as your "default" via the AWS CLI. If you'd like to use different credentials, you can modify the code below to add `aws_access_key_id` and `aws_secret_access_key` arguments to the setup function.


In [ ]:
import boto3
import logging

logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
logger = logging.getLogger(__name__)

MODEL_ID = "amazon.nova-reel-v1:0"

# Set default region and credentials.
boto3.setup_default_session(
    region_name="us-east-1"
)

logger.info("AWS SDK session defaults have been set.")

## Invoking the model

Generating a video takes some time - approximately 3.5 minutes to produce a 6 second video. To accomodate this execution time, the Bedrock Runtime introduces a new asynchronous invocation API. Calling `start_async_invoke()` creates a new invocation job. When the job completes, Bedrock automatically saves the generated video to an S3 bucket you specify.

### Image-to-Video

You can also generate videos by providing an initial starting image and a text prompt. For best results, the text prompt should describe the image and also provide details about the desired action and camera movement you'd like the video to have. Modify the `s3_destination_bucket`, `input_image_path`, and `video_prompt` variables at the start of the code below and then run the cell to start generating your video.



<div style="display: flex; justify-content: space-between;">
    <div style="width: 31%">
        <p align="center">
            <video alt="example_text_to_video" controls style="padding: 4px" >
                <source src="../videos/snow_1.mp4" type="video/mp4" >
            </video>
            <br>
            <em>First video</em>
        </p>
    </div>
    <div style="width: 31%;">
        <p align="center">
            <img src="../images/snow_last_frame.jpg" width="100%" style="padding: 4px">
            <br>
            <em>Last frame image</em>
        </p>
    </div>
    <div style="width: 31%">
        <p align="center">
            <video alt="example_text_to_video" controls style="padding: 4px" >
                <source src="../videos/snow_merged.mp4" type="video/mp4" >
            </video>
            <br>
            <em>Merged video</em>
        </p>
    </div>
</div>

In [ ]:
import json
import random
import amazon_video_util

"""
IMPORTANT: Modify the S3 destination (s3_destination_bucket) and video prompt (video_prompt) below.
"""

# Specify an S3 bucket for the video output.
s3_destination_bucket = "nova-videos"  # Change this to a unique bucket name.

# Specify your video generation prompt. Phrase your prompt as a summary rather than a command. Maximum 512 characters.
video_prompt = "First person view walking through light snowfall in a forest, sunlight through trees, dolly forward, cinematic"

"""
STOP: You should not have to modify anything below this line.
"""

# Set up the S3 client.
s3_client = boto3.client("s3")

# Create the S3 bucket.
s3_client.create_bucket(Bucket=s3_destination_bucket)

# Create the Bedrock Runtime client.
bedrock_runtime = boto3.client("bedrock-runtime")

model_input = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": video_prompt
        },
    "videoGenerationConfig": {
        "durationSeconds": 6, 
        "fps": 24,
        "dimension": "1280x720",
        "seed": random.randint(0, 2147483646),
    },
}

try:
    # Start the asynchronous video generation job.
    invocation = bedrock_runtime.start_async_invoke(
        modelId="amazon.nova-reel-v1:0",
        modelInput=model_input,
        outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{s3_destination_bucket}"}},
    )

    # This will be used by other cells in this notebook.
    invocation_arn = invocation["invocationArn"]

    # Pretty print the response JSON.
    logger.info("\nResponse:")
    logger.info(json.dumps(invocation, indent=2, default=str))

    # Save the invocation details for later reference. Helpful for debugging and reporting feedback.
    amazon_video_util.save_invocation_info(invocation, model_input)

except Exception as e:
    logger.error(e)

## Checking the status of generation jobs

We've provided a set of utility functions in the `amazon_video_util.py` script. One of these functions will automatically download a job if it is completed or monitor it while it is in-progress. The `invocation_arn` is defined in the code cell above and passed in below.



In [ ]:
# Monitor and download the first video generation
local_file_path_1 = amazon_video_util.monitor_and_download_video(invocation_arn, "output")

### Last Frame Extraction

Below we will first extract the last frame from the first video to use as a starting point for the second videos generation. This will allow for extending the original video past the 6 second limit.

In [ ]:
import os
from PIL import Image
from datetime import datetime

# Define and create an output directory with a unique name.
generation_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_directory = f"./output/{generation_id}"
os.makedirs(output_directory, exist_ok=True)

# Extract the last frame from the video
output_path = f"{output_directory}/last_frame.png"
amazon_video_util.extract_last_frame(local_file_path_1, output_path)

# Display the last frame
with Image.open(output_path) as last_frame:
    last_frame.load()
    display(last_frame)

### Generating the second video
Using the frame above we can now extend the first video. Make sure the prompt maintains the core elements and objects from the first generation to get the best result.

In [ ]:
import base64
from io import BytesIO

"""
IMPORTANT: Modify the video prompt (video_prompt) below.
"""

# Specify your video generation prompt. Phrase your prompt as a summary rather than a command. Maximum 512 characters.
video_prompt = "First person view entering a clearing with heavy snowfall, sun creating diamond-like sparkles, continuing dolly forward, cinematic"

"""
STOP: You should not have to modify anything below this line.
"""

# Load the input image as a Base64 string.
buffered = BytesIO()
last_frame.save(buffered, format="JPEG")
input_image_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

model_input = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {
        "text": video_prompt,
        "images": [
            {
                "format": "png",  # May be "png" or "jpeg"
                "source": {"bytes": input_image_base64},
            }
        ],
    },
    "videoGenerationConfig": {
        "durationSeconds": 6,  # 6 is the only supported value currently.
        "fps": 24,  # 24 is the only supported value currently.
        "dimension": "1280x720",  # "1280x720" is the only supported value currently.
        "seed": random.randint(
            0, 2147483648
        ),  # A random seed guarantees we'll get a different result each time this code runs.
    },
}

try:
    # Start the asynchronous video generation job.
    invocation = bedrock_runtime.start_async_invoke(
        modelId="amazon.nova-reel-v1:0",
        modelInput=model_input,
        outputDataConfig={
            "s3OutputDataConfig": {"s3Uri": f"s3://{s3_destination_bucket}"}
        },
    )

    # This will be used by other cells in this notebook.
    invocation_arn = invocation["invocationArn"]

    # Pretty print the response JSON.
    logger.info("\nResponse:")
    logger.info(json.dumps(invocation, indent=2, default=str))

    # Save the invocation details for later reference. Helpful for debugging and reporting feedback.
    amazon_video_util.save_invocation_info(invocation, model_input)

except Exception as e:
    logger.error(e)

### Monitor the second generation
As with the first generation, we will monitor and download the video.

In [ ]:
local_file_path_2 = amazon_video_util.monitor_and_download_video(invocation_arn, "output")

### Stitch the videos
We've provided another utility function in `amazon_video_util` that can be used to stitch the two videos together into a single video.

In [ ]:
# Create output path for merged video
output_path = f"{output_directory}/merged_video.mp4"

# Stitch the two video generations together
amazon_video_util.stitch_videos(local_file_path_1, local_file_path_2, output_path)

### Viewing final video
Finally we can now view our extended video. You can try varying the seed and prompt of the second generation to get the best result.

In [ ]:
from IPython.display import Video
Video(output_path, embed=True)